El archivo homicidios viene con dos hojas de excel que son victimas y hechos, abro cada una en un notebook diferente para modificar y tranformar los valores que se necesite

La información que contiene la hoja de datos hechos es hechos de transito con vehículos en movimiento 

In [ ]:
pip install geopy

In [3]:
import json
import pandas as pd
import ast
from datetime import datetime, time
from geopy.geocoders import Nominatim

In [4]:
# Abro libro hechos del archivo homicidios y lo dejo como dataframe de csv

hechos_original = r"C:\Users\Usuario\Henry\PI2_DA\Consigna\hechos_original.csv"


In [5]:
df_hechos = pd.read_csv(hechos_original)

In [ ]:
df_hechos.info()

In [ ]:
df_hechos.head()

Se puede ver que este conjunto de datos cuenta con 21 columnas reterentes a las caracteristicas del evento como la fecha el numero de victimas, el lugar, y la ubicación que se podría normalizar de mejor forma. inicialmente voy a dejar una columna con la fecha y la hora para borrar la columnas que contienen esta información

In [408]:
# Borro columna HH que ya esta en hora

df_hechos = df_hechos.drop(columns=['HH'])


In [ ]:
df_hechos.info()

In [410]:
df_hechos.size
# es corto

13920

In [225]:
# De entrada borro na
# df_hechos.dropna()
# No puedo borrar los na por que me borra todo, debe haber una columna que tiene en todas las casillas ne

In [411]:
# Borro duplicados que si puedo borrar

df_hechos = df_hechos.drop_duplicates()

In [412]:
df_hechos.size
# No hay duplicados pero no sobra

13920

In [413]:
# Me aseguro que toda la columna de fecha tenga el mismo formato YY-MM-DD 

df_hechos['FECHA'] = pd.to_datetime(df_hechos['FECHA'], format='%Y-%m-%d')


In [ ]:
df_hechos.info()

In [415]:
# Dado que esa columna ya tiene la info de toda la fecha borro columnas de año, mes y dia

columnas_borrar = ['AAAA', 'MM', 'DD']

df_hechos = df_hechos.drop(columns = columnas_borrar, axis=1)


In [ ]:
df_hechos.info()

Ya me ocupe de la hora, ahora revisaré todas las columnas de uicación

In [157]:
# Averiguando en Chatgpt veo la función para devolver la dirección con las coordenadas
# para eso tomo una fila y hago el ejemplo

In [417]:
def obtener_direccion(COORDENADAS):
    latitud, longitud = COORDENADAS
    geolocalizador = Nominatim(user_agent="mi_aplicacion")
    ubicacion = geolocalizador.reverse(COORDENADAS, language='es')
    return ubicacion.address

In [418]:
# tomo ejemplo de Coordenadas geográficas, suponiendo que posx es latitud y posy altitud
LATITUD = -34.68757022
LONGITUD = -58.47533969

In [419]:
print("Dirección:", direccion)

#Lo hice pero se demora demasiado así que usaré la dirección del df de la consigna

Dirección: 6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina


In [78]:
# En realidad funcionó!!!, la dirección que me da el sistema es
# Dirección: 6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina
# Tambien cuadra numer de la comuna

In [420]:
# ya sabiendo que si son las coordenadas, Cambio el nombre de las columnas por latitud y longitud, las junto en una que se 
# llame coordenadas y con esa columna ya tendría toda la información

df_hechos = df_hechos.rename(columns={'pos x': 'LONGITUD'})
df_hechos = df_hechos.rename(columns={'pos y': 'LATITUD'})


In [ ]:
df_hechos.head()

In [422]:
# Voy a verificar que en las columnas latitud y longitud no hayan nulos

df_hechos = df_hechos.dropna(subset=['LONGITUD'])


In [423]:
# Voy a verificar que en las columnas latitud y longitud no hayan nulos

df_hechos = df_hechos.dropna(subset=['LATITUD'])

In [424]:
df_hechos.size

11832

In [425]:
# Me aseguro que longitud y latitud Sean numericos
df_hechos['LONGITUD'] = pd.to_numeric(df_hechos['LONGITUD'], errors='coerce')
df_hechos['LATITUD'] = pd.to_numeric(df_hechos['LATITUD'], errors='coerce')


In [426]:
df_hechos.size

11832

In [427]:
# Junto las 2 columnas en una que se llame COORDENADAS, por que la función que encontré solo tiene un argumento

df_hechos['COORDENADAS'] = df_hechos.apply(lambda row: (row['LONGITUD'], row['LATITUD']), axis=1)



In [ ]:
df_hechos.info()

In [ ]:
df_hechos.head()

In [430]:
# con la columna coordenadas tengo toda la descripción de la ubicación, es importante comuna, 
# Me aseguro que las columnas altitud y latitud sean numericas

df_hechos['ALTITUD'] = df_hechos['LATITUD'].apply(pd.to_numeric, errors='coerce')

df_hechos['LONGITUD'] = df_hechos['LONGITUD'].apply(pd.to_numeric, errors='coerce')


In [ ]:
df_hechos.head()

In [431]:
# Con las coordenada ya tengo toda la info y puedo graficar en powebi asi que borro las que no interesan tanto

Columnas_borrar = ['TIPO_DE_CALLE', 'Calle', 'Altura',  'XY (CABA)', 'LONGITUD', 'ALTITUD',	'LUGAR_DEL_HECHO', 'LATITUD', 'PARTICIPANTES']


In [432]:
df_hechos =  df_hechos.drop(Columnas_borrar, axis=1)


In [ ]:
df_hechos.info()

In [438]:
# Voy a renombrar Cruce y Dirección Normalizada por que todo esta en mayusculas asi que no se ve bien

df_hechos = df_hechos.rename(columns={'Cruce': 'CRUCE'})
df_hechos = df_hechos.rename(columns={'Dirección Normalizada': 'DIRECCION'})


In [ ]:
df_hechos.info()

In [ ]:
df_hechos.head()

In [ ]:
# Pienso que con esta info es suficiente para la primera parte pero ahora toca limpiar
# la info dado que yo nunca borre na porque borraba toda la info


In [442]:
df_hechos.to_parquet('df_hechos.parquet')
